In [1]:
import pandas as pd

df = pd.read_excel('84사전최종.xlsx', index_col=0)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, 0 to 85
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   code         86 non-null     int64 
 1   information  86 non-null     object
dtypes: int64(1), object(1)
memory usage: 2.0+ KB


In [4]:
df.head()

,code,information
0,8401,"Nuclear reactors; fuel elements (cartridges), ..."
1,8402,Steam or other vapour generating boilers (othe...
2,8403,Central heating boilers other than those of he...
3,8404,Auxiliary plant for use with boilers of headin...
4,8405,"Producer gas or water gas generators, with or ..."


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
headings = []
keyword = []
cnt = 0
for i in df['information']:

    # 텍스트 전처리: 소문자 변환, 토큰화, 불용어 제거
    tokens = word_tokenize(i.lower())
    stop_words = set(stopwords.words('english'))  # NLTK의 영어 불용어 목록을 사용합니다. 해당 언어에 맞게 변경 가능
    
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    
    # TF-IDF 벡터화
    tfidf = TfidfVectorizer(ngram_range=(1, 2))
    tfidf_matrix = tfidf.fit_transform([' '.join(filtered_tokens)])
    
    # TF-IDF 행렬에서 가장 중요한 단어 추출
    feature_names = tfidf.get_feature_names_out()
    
    # TF-IDF 값이 높은 단어 순으로 정렬
    dense = tfidf_matrix.todense()
    keywords = sorted(zip(dense[0].tolist()[0], feature_names), reverse=True)
    
    # 상위 키워드를 출력하거나 리스트로 가져올 수 있습니다.
    top_keywords = [word for score, word in keywords[:50]]  # 상위 5개의 키워드를 가져옴
    print(f"상위 키워드:{df['code'][cnt]}", top_keywords)

    headings.append(df['code'][cnt])
    keyword.append(top_keywords)
    cnt += 1

# 어근추출 추가

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

headings = []
keyword = []
cnt = 0

# Initialize WordNet Lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

for i in df['information']:
    # 텍스트 전처리: 소문자 변환, 토큰화, 불용어 제거, 어근 추출
    tokens = word_tokenize(i.lower())
    filtered_tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]

    # TF-IDF 벡터화
    tfidf = TfidfVectorizer(ngram_range=(1, 2))
    tfidf_matrix = tfidf.fit_transform([' '.join(filtered_tokens)])

    # TF-IDF 행렬에서 가장 중요한 단어 추출
    feature_names = tfidf.get_feature_names_out()

    # TF-IDF 값이 높은 단어 순으로 정렬
    dense = tfidf_matrix.todense()
    keywords = sorted(zip(dense[0].tolist()[0], feature_names), reverse=True)

    # 상위 키워드를 출력하거나 리스트로 가져올 수 있습니다.
    top_keywords = [word for score, word in keywords[:50]]  # 상위 5개의 키워드를 가져옴
    print(f"{df['code'][cnt]} 키워드:", top_keywords)

    headings.append(df['code'][cnt])
    keyword.append(top_keywords)
    cnt += 1

# 어근추출 + 복수형 단수화

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')  # pos_tag를 위한 데이터 다운로드

headings = []
keyword = []
cnt = 0

# Initialize WordNet Lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

for i in df['information']:
    # 텍스트 전처리: 소문자 변환, 토큰화, 불용어 제거
    tokens = word_tokenize(i.lower())
    tagged_tokens = nltk.pos_tag(tokens)  # 단어 토큰에 품사 태깅 추가

    # 품사를 고려하여 단어의 원형 복원 (명사와 동사만 고려)
    filtered_tokens = []
    for word, tag in tagged_tokens:
        if (tag.startswith('NN') or tag.startswith('VB')) and word not in stop_words:  # 명사 또는 동사인 경우
            lemma = lemmatizer.lemmatize(word, pos='n' if tag.startswith('NN') else 'v')
            filtered_tokens.append(lemma)
        elif word.isalpha() and word not in stop_words:
            filtered_tokens.append(word)

    # TF-IDF 벡터화
    tfidf = TfidfVectorizer(ngram_range=(1, 3))
    tfidf_matrix = tfidf.fit_transform([' '.join(filtered_tokens)])

    # TF-IDF 행렬에서 가장 중요한 단어 추출
    feature_names = tfidf.get_feature_names_out()

    # TF-IDF 값이 높은 단어 순으로 정렬
    dense = tfidf_matrix.todense()
    keywords = sorted(zip(dense[0].tolist()[0], feature_names), reverse=True)

    # 상위 키워드를 출력하거나 리스트로 가져올 수 있습니다.
    top_keywords = [word for score, word in keywords[:100]]  # 상위 5개의 키워드를 가져옴
    print(f"{df['code'][cnt]} 키워드:", top_keywords)

    headings.append(df['code'][cnt])
    keyword.append(top_keywords)
    cnt += 1

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


8401 키워드: ['reactor', 'water', 'uranium', 'fuel', 'element', 'apparatus', 'fuel element', 'nuclear', 'nuclear reactor', 'element cartridge', 'cartridge', 'heavy water', 'heavy', 'fuel element cartridge', 'neutron', 'material', 'fissile', 'use', 'part', 'metal', 'may', 'uranium hexafluoride', 'type', 'hexafluoride', 'compound', 'also', 'isotopic', 'general', 'fit', 'fertile', 'tube', 'thermal', 'steel', 'separation', 'production heavy water', 'production heavy', 'production', 'moderator', 'isotope', 'include', 'fraction', 'example', 'etc', 'enrichment', 'classify', 'base', 'special', 'source', 'shield', 'sheath', 'separate', 'rod', 'reaction', 'produce', 'plate', 'part nuclear reactor', 'part nuclear', 'often', 'obtain', 'natural', 'molten', 'metallic', 'inside', 'inert metal', 'inert', 'hydrogen', 'however', 'high', 'heading', 'head', 'graphite', 'generally', 'gaseous', 'gas', 'form', 'fissile fertile', 'exchange', 'enrich', 'element cartridge nuclear', 'diffusion', 'different', 'deute

In [ ]:
data = {'hs_code': headings, 'keyword': keyword}
df = pd.DataFrame(data)

In [ ]:
df.head(10)

In [ ]:
df.to_excel('84keywords_100.xlsx')